# Run XQuery in Jupyter Notebooks via eXist, by Joe Wicentowski (Jan 5, 2020)

Requirements: 

- Anaconda Python (version 2019-10), Jupyter Lab (1.1.14)
- eXist-db (5.1.1), with the [Atom editor support](https://github.com/eXist-db/atom-editor-support) package installed (this can also installed by using the [Atom editor package for eXistdb](https://atom.io/packages/existdb) or by directly downloading the required xar from )
- The final two cells assume that https://github.com/djbpitt/neh_07_tei-view is installed in eXist

Goals:

- To execute XQuery scripts against an eXist server from Jupyer Notebooks
- To return query results to Jupyter Notebooks at least as well as they're returned by eXide and Atom
- To easily switch between XQuery serialization methods (xml, html, text, json, and adaptive)

Limitations:

- Queries are constructed in python strings, so no syntax highlighting is applied to the source query 
- Due to the way the query handler evaluates queries, they inherit certain context from the 
  Atom support handler, so certain prolog option declarations, such as boundary-space, context item, 
  item-separator, output, etc. will be overridden by the query handler. (The same limitations apply 
  to eXide and Atom though. Note that we opted for Atom's handler since it returns all results to the 
  query; eXide's stores results in a session parameter, where they must be fetched with subsequent HTTP 
  requests, one per result.)
- Currently all results are returned with the "indent: yes" serialization option, though a future
  version should allow indentation to be specified. 

Credits:

- Thanks to RHD for demonstrating how to use Jupyter to send HTTP requests to eXist
- Thanks to Jonathan Robie for sharing his experience doing something similar with BaseX

References: 

- XQuery Serialization methods: https://www.w3.org/TR/xslt-xquery-serialization-31/
- Jonathan Robie's writeup http://jonathanrobie.biblicalhumanities.org/blog/2017/12/08/jupyter-tutorial/
- Ronald Dekker's experiment https://github.com/Pittsburgh-NEH-Institute/Institute-Materials-2020/blob/master/Python_ExistDB_REST/ExistDB_and_REST_experiment.ipynb
- Atom editor support package query handler: 
  - https://github.com/eXist-db/atom-editor-support/blob/master/results.xql
  - https://github.com/eXist-db/atom-editor-support/blob/master/controller.xql#L34-L79
- Syntax highlighting: https://stackoverflow.com/questions/35950433/getting-pygments-to-work-for-my-cell-results


In [162]:
# This cell defines a Python function that can be used in subsequent cells to send requests to eXist-db

import requests
from pygments import highlight
from pygments.lexers import XQueryLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import display, HTML

# Send the query to eXist-db, serializing the results using a standard output method
# Standard serialization methods: html, json, xml, adaptive, text
# The html output method is rendered as HTML; for others, syntax highlighting is applied
def run_xquery(query, output_method):
  url = 'http://localhost:8080/exist/apps/atom-editor/execute'
  parameters = {
    'qu': query,
    'output': output_method
  }
  r = requests.post(url, data = parameters)
  if output_method == 'html':
    display(HTML(r.text))
  else:
    display(HTML('<style>{pygments_css}</style>'.format(pygments_css=HtmlFormatter().get_style_defs('.highlight'))))
    display(HTML(data=highlight(r.text, XQueryLexer(), HtmlFormatter())))

# If syntax highlighting ever slows down results, replace with:
#    print(r.text)

In [163]:
# This cell demonstrates how to set up an XQuery and send it to eXist: 
# (1) write your XQuery in a string (using three quotes to create a multiline string, if needed)
# (2) assign the string to a variable, e.g., query
# (3) pass the query string the run_xquery function, which also needs a serialization method
# Notice how 'html' is rendered, rather than displayed as source. If needed, a future version could 
# distinguish between html serialization and rendering, allowing 

query = """

xquery version "3.1";

<ul>{(1 to 10) ! <li>{.}</li>}</ul>

"""

run_xquery(query, 'xml')

run_xquery(query, 'html')

run_xquery(query, 'text')


In [159]:
# Adaptive serialization is unique in that it can represent nearly all XDM types

query = """

xquery version "3.1";

map { 
    "items": 
        array { 
            1 to 10, 
            current-date(),
            current-date#0,
            attribute date { current-date() }
        } 
}

"""


run_xquery(query, 'adaptive')

run_xquery(query, 'json')

# xml serialization gives up on displaying non-xml XDM types completely
run_xquery(query, 'xml')

In [160]:
# This cell demonstrates how database resources can be accessed.
# Note: Assumes the package https://github.com/djbpitt/neh_07_tei-view is installex in eXist.

query = """

xquery version "3.1";

let $collection := "/db/apps/neh_07_tei-view/xml"
let $resources := xmldb:get-child-resources($collection) => sort()
return
    <div>
        <h2>{$collection}</h2>
        <p>The collection <strong>{$collection}</strong> contains {count($resources)} resources</p>
        <ol>{
            $resources ! <li>{.}</li>
        }</ol>
    </div>

"""

results = run_xquery(query, 'html')

In [161]:
# This cell builds on the previous example, showing how TEI can be transformed into HTML.

query = """

xquery version "3.1";

declare default element namespace "http://www.w3.org/1999/xhtml";

declare namespace tei = "http://www.tei-c.org/ns/1.0";

declare function local:dispatch($node as node()) as item()* {
    typeswitch($node)
        case text() return $node
        case element(tei:ab) return local:ab($node)
        case element(tei:bibl) return local:bibl($node)
        case element(tei:body) return local:body($node)
        case element(tei:cit) return local:cit($node)
        case element(tei:closer) return local:closer($node)
        case element(tei:date) return local:date($node)
        case element(tei:div) return local:div($node)
        case element(tei:emph) return local:emph($node)
        case element(tei:head) return local:head($node)
        case element(tei:interp) return local:interp($node)
        case element(tei:lb) return local:lb($node)
        case element(tei:name) return local:name($node)
        case element(tei:note) return local:note($node)
        case element(tei:opener) return local:opener($node)
        case element(tei:p) return local:p($node)
        case element(tei:pb) return local:pb($node)
        case element(tei:persName) return local:persName($node)
        case element(tei:placeName) return local:placeName($node)
        case element(tei:q) return local:q($node)
        case element(tei:quote) return local:quote($node)
        case element(tei:ref) return local:ref($node)
        case element(tei:rs) return local:rs($node)
        case element(tei:salute) return local:salute($node)
        case element(tei:sic) return local:sic($node)
        case element(tei:signed) return local:signed($node)
        case element(tei:soCalled) return local:soCalled($node)
        case element(tei:title) return local:title($node)
        default return local:passthru($node)
};

declare function local:ab($node as element(tei:ab)) as element() {
    <p>{local:passthru($node)}</p>};

declare function local:bibl($node as element(tei:bibl)) as item()+ {
    (: only example is attribution immediately after a quote, so inline:)
    (' (', local:passthru($node), ')')
};

declare function local:body($node as element(tei:body)) as element() {
    <section id="body">{local:passthru($node)}</section>};

declare function local:cit($node as element(tei:cit)) as element() {
    (: only example is inline :)
    <q>{local:passthru($node)}</q>};

declare function local:closer($node as element(tei:closer)) as element() {
    <span class="closer">{local:passthru($node)}</span>};

declare function local:date($node as element(tei:date)) as element() {
    <span class="date">{local:passthru($node)}</span>};

declare function local:div($node as element(tei:div)) as element() {
    <div>{local:passthru($node)}</div>};

declare function local:emph($node as element(tei:emph)) as element() {
    <em>{local:passthru($node)}</em>};

declare function local:head($node as element(tei:head)) as element() {
    (: TODO: currently just <h1>, need to adjust GI to depth of nesting :)
    <h1>{local:passthru($node)}</h1>};

declare function local:interp($node as element(tei:interp)) as element() {
    <span class="interp">{local:passthru($node)}</span>};

declare function local:lb($node as element(tei:lb)) as element() {
    (: empty :)
    <lb/>
};

declare function local:name($node as element(tei:name)) as element() {
    <span class="name">{local:passthru($node)}</span>};

declare function local:note($node as element(tei:note)) as element() {
    (: only instance is inline parenthetical phrase from the original:)
    (' (', local:passthru($node), ')')};

declare function local:opener($node as element(tei:opener)) as element() {
    <span class="opener">{local:passthru($node)}</span>};

declare function local:p($node as element(tei:p)) as element() {
    <p>{local:passthru($node)}</p>};

declare function local:pb($node as element(tei:pb)) as element() {
    (: empty :)
    <pb/>
};

declare function local:persName($node as element(tei:persName)) as element() {
    <span class="persName">{local:passthru($node)}</span>};

declare function local:placeName($node as element(tei:placeName)) as element() {
    <span class="placeName">{local:passthru($node)}</span>};

declare function local:q($node as element(tei:q)) as element() {
    <q>{local:passthru($node)}</q>};

declare function local:quote($node as element(tei:quote)) as element() {
    <q>{local:passthru($node)}</q>};

declare function local:ref($node as element(tei:ref)) as element() {
    <a href="{@source}">{local:passthru($node)}</a>};

declare function local:rs($node as element(tei:rs)) as element() {
    (: not yet functional; need to create working @href values :)
    <a href="#ref">{local:passthru($node)}</a>};

declare function local:salute($node as element(tei:salute)) as element() {
    <span class="salute">{local:passthru($node)}</span>};

declare function local:sic($node as element(tei:sic)) as element() {
    <span class="sic">{local:passthru($node)}</span>};

declare function local:signed($node as element(tei:signed)) as element() {
    <span class="signed">{local:passthru($node)}</span>};

declare function local:soCalled($node as element(tei:soCalled)) as element() {
    <span class="soCalled">{local:passthru($node)}</span>};

declare function local:title($node as element(tei:title)) as element() {
    if ($node/parent::*) 
    then <h1>{local:passthru($node)}</h1>
    else <cite>{local:passthru($node)}</cite>};

declare function local:passthru($node as node()) as item()* {
    for $child in $node/node() 
    return 
        local:dispatch($child)
};

let $doc := doc("/db/apps/neh_07_tei-view/xml/ablackghost_johnbull_1838.xml")
return
    local:passthru($doc)
"""

run_xquery(query, 'html')